In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import tensorflow as tf
import tensorflow.keras as keras

# Load data

In [4]:
fashion_mnist = keras.datasets.fashion_mnist.load_data()

# Train Test Split

In [5]:
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

In [7]:
tf.keras.backend.clear_session()
tf.random.set_seed(2024)

# Build Model

In [13]:
import keras_tuner as kt

def build_model(hp):
    """
    Takes hyperparameter as argument and returns a model
    """

    num_hidden_layers = hp.Int("num_hidden_layers", min_value = 0, max_value = 0, default = 2)
    num_neurons = hp.Int("num_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value = 1e-4, max_value=1e-2, sampling='log')
    optimizer = hp.Choice("optimizer", values=["sgd","adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Flatten())

    for _ in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(num_neurons, activation="relu")) # activation can also come from hp
    
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss = "sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

# Optimize Model

In [17]:
bayesian_search_tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=5,
    overwrite=True,
    directory = "my_fashion_mnist_hp_tuning",
    project_name = "bayesian_search",
    seed=20240
)

In [18]:
bayesian_search_tuner.search(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Trial 5 Complete [00h 00m 23s]
val_accuracy: 0.8438000082969666

Best val_accuracy So Far: 0.8438000082969666
Total elapsed time: 00h 02m 11s


# Select best Models

In [20]:
top3_models = bayesian_search_tuner.get_best_models(num_models=3)
best_model = top3_models[0]
best_model

In [24]:
top3_params = bayesian_search_tuner.get_best_hyperparameters(num_trials=3)
top3_params[0].values

{'num_hidden_layers': 0,
 'num_neurons': 158,
 'learning_rate': 0.0015019300136883287,
 'optimizer': 'sgd'}

In [25]:
best_trial = bayesian_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 4 summary
Hyperparameters:
num_hidden_layers: 0
num_neurons: 158
learning_rate: 0.0015019300136883287
optimizer: sgd
Score: 0.8438000082969666


In [26]:
best_trial.metrics.get_last_value("val_accuracy")

0.8438000082969666

In [27]:
best_model.fit(X_train_full, y_train_full, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 2s 1ms/step - loss: 219.9636 - accuracy: 0.7977
Epoch 2/10
1875/1875 [==============================] - 2s 1ms/step - loss: 219.3078 - accuracy: 0.7982
Epoch 3/10
1875/1875 [==============================] - 2s 1ms/step - loss: 221.6801 - accuracy: 0.7978
Epoch 4/10
1875/1875 [==============================] - 2s 1ms/step - loss: 217.7541 - accuracy: 0.7992
Epoch 5/10
1875/1875 [==============================] - 3s 1ms/step - loss: 215.5266 - accuracy: 0.8027
Epoch 6/10
1875/1875 [==============================] - 3s 1ms/step - loss: 212.6148 - accuracy: 0.8028
Epoch 7/10
1875/1875 [==============================] - 2s 1ms/step - loss: 212.0956 - accuracy: 0.8044
Epoch 8/10
1875/1875 [==============================] - 2s 1ms/step - loss: 208.5959 - accuracy: 0.8033
Epoch 9/10
1875/1875 [==============================] - 2s 1ms/step - loss: 216.8687 - accuracy: 0.8012
Epoch 10/10
1875/1875 [==============================] - 2s 1ms/

In [29]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (32, 784)                 0         
                                                                 
 dense (Dense)               (32, 10)                  7850      
                                                                 
Total params: 7850 (30.66 KB)
Trainable params: 7850 (30.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print(test_loss, test_accuracy)